In [29]:
import pandas as pd
import os
import sys

#### Data: http://www2.informatik.uni-freiburg.de/~cziegler/BX/

In [30]:
folder = 'BX-CSV-Dump'
ratings = 'BX-Book-Ratings.csv'
books = 'BX-Books.csv'
users = 'BX-Users.csv'

In [31]:
df_ratings = pd.read_csv(os.path.join(folder, ratings), sep=';')

In [32]:
df_ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [33]:
# there seems to be some empty fields after the desired number of fields(8), so we use bad lines func skip them
df_books = pd.read_csv(os.path.join(folder, books), sep=';', error_bad_lines=False, low_memory=False)
len(df_books)

Skipping line 6452: expected 8 fields, saw 9
Skipping line 43667: expected 8 fields, saw 10
Skipping line 51751: expected 8 fields, saw 9
Skipping line 92038: expected 8 fields, saw 9
Skipping line 104319: expected 8 fields, saw 9
Skipping line 121768: expected 8 fields, saw 9
Skipping line 144058: expected 8 fields, saw 9
Skipping line 150789: expected 8 fields, saw 9
Skipping line 157128: expected 8 fields, saw 9
Skipping line 180189: expected 8 fields, saw 9
Skipping line 185738: expected 8 fields, saw 9
Skipping line 209388: expected 8 fields, saw 9
Skipping line 220626: expected 8 fields, saw 9
Skipping line 227933: expected 8 fields, saw 11
Skipping line 228957: expected 8 fields, saw 10
Skipping line 245933: expected 8 fields, saw 9
Skipping line 251296: expected 8 fields, saw 9
Skipping line 259941: expected 8 fields, saw 9
Skipping line 261529: expected 8 fields, saw 9



271360

In [34]:
# i have capture the bad line no manually, and have appended them to the dataframe
f = open(os.path.join(folder, books))
lines = f.readlines()
bad_line_nos = [6452, 43667, 51751, 92038, 104319, 121768, 144058, 
                150789, 157128, 180189, 185738, 209388, 220626, 27933,
                28957, 245933, 251296, 259941, 261529]
skipped_rows = 0

for line in bad_line_nos:
    parsed_row = lines[line].split(";")
    df_books.loc[len(df_books) + 1] = parsed_row[:8]

print len(df_books)

271379


In [35]:
df_books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [36]:
df_users = pd.read_csv(os.path.join(folder, users), sep=';', error_bad_lines=False)
df_users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


BX-Users
Contains the users. Note that user IDs (`User-ID`) have been anonymized and map to integers. Demographic data is provided (`Location`, `Age`) if available. Otherwise, these fields contain NULL-values.

BX-Books
Books are identified by their respective ISBN. Invalid ISBNs have already been removed from the dataset. Moreover, some content-based information is given (`Book-Title`, `Book-Author`, `Year-Of-Publication`, `Publisher`), obtained from Amazon Web Services. Note that in case of several authors, only the first is provided. URLs linking to cover images are also given, appearing in three different flavours (`Image-URL-S`, `Image-URL-M`, `Image-URL-L`), i.e., small, medium, large. These URLs point to the Amazon web site.

BX-Book-Ratings
Contains the book rating information. Ratings (`Book-Rating`) are either explicit, expressed on a scale from 1-10 (higher values denoting higher appreciation), or implicit, expressed by 0.

#### all the data files have been loaded, but ratings file will be mostly used to make recommendation. Other files, user info or book info at a later stage can  considered for including as metrics to make recommendation.
#### for now we will concentrate on ratings file

In [37]:
df_ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [38]:
# freq_user location
freq_user = df_ratings.groupby(['User-ID']).agg({'Book-Rating': 'count'})
freq_user = freq_user.rename(columns={'Book-Rating': 'count'})
list_ids = freq_user.sort_values(['count'], ascending=False).head(5)
df_users[df_users['User-ID'].isin(list_ids.index)]

,User-ID,Location,Age
11675,11676,"n/a, n/a, n/a",NaN
35858,35859,"duluth, minnesota, usa",NaN
98390,98391,"morrow, georgia, usa",52.0
153661,153662,"ft. stewart, georgia, usa",44.0
198710,198711,"little canada, minnesota, usa",62.0


In [39]:
# freq_book
freq_books = df_ratings.groupby(['ISBN']).agg({'Book-Rating': 'count'})
freq_books = freq_books.rename(columns={'Book-Rating': 'count'})
list_books = freq_books.sort_values(['count'], ascending=False).head(5)
df_books[df_books['ISBN'].isin(list_books.index)]['Book-Title']

26                                          Wild Animus
408                           The Lovely Bones: A Novel
522                 The Red Tent (Bestselling Backlist)
748                                   The Da Vinci Code
1105    Divine Secrets of the Ya-Ya Sisterhood: A Novel
Name: Book-Title, dtype: object

#### let's get into some algorithms

In [44]:
def minkwosian_distance(rating1, rating2, r):
    rating1 = rating1.rename(columns={'Book-Rating': 'rating1'})
    rating2 = rating2.rename(columns={'Book-Rating': 'rating2'})
    common_ISBNs = rating1.index.intersection(rating2.index)
    users_ratings = pd.concat([rating1.loc[common_ISBNs], rating2.loc[common_ISBNs]], axis=1)
    # pick only non zero ratings, that way we wil avoid not rated item
    users_ratings = users_ratings[users_ratings['rating1'] > 0]
    users_ratings = users_ratings[users_ratings['rating2'] > 0]
    
    users_ratings['manhatan_dis'] = pow(abs(users_ratings['rating1'] - users_ratings['rating2']), r)

    return pow(users_ratings['manhatan_dis'].sum(), 1/float(r))

#### checking how much the frequent users (5) similarity measures

In [45]:
def fil_user_info(r_df, user_n):
    r_df = r_df[r_df['User-ID'] == user_n]
    return r_df[['ISBN', 'Book-Rating']].set_index('ISBN')

In [46]:
u_rating1 = fil_user_info(df_ratings, 11676)
u_rating2 = fil_user_info(df_ratings, 35859)
minkwosian_distance(u_rating1, u_rating2, 2)

827


28.75760768909681

In [43]:
df_ratings['User-ID'].nunique()

105283